In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [6]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 3

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        #print(y_pred.shape)
        #show_sample_batch(sample_batch, agent_id)

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


tensor(593.2542, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(515.4708, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(338.8319, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(435.2574, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(307.8954, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(339.3917, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(237.1464, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(286.2760, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(234.6000, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(203.8491, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(193.2150, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(181.6808, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(138.3428, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(132.8286, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(107.6171, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(138.9130, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(85.2399, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(91.1226,

KeyboardInterrupt: 

In [14]:
torch.save(model, './models/6epochmodel.pt')

In [4]:
model = torch.load('./models/6epochmodel.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [6]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                


         ID                  v1                  v2                  v3  \
0     10002  1763.3968505859375   412.9548645019531  1772.8170166015625   
1     10015   725.8516845703125     1222.8857421875   720.3778686523438   
2     10019   572.6814575195312   1245.853271484375   572.7365112304688   
3     10028      1734.490234375    392.561279296875   1745.483154296875   
4      1003    2093.14892578125   659.0765991210938    2093.48193359375   
...     ...                 ...                 ...                 ...   
3195   9897   258.7663269042969   804.1239624023438   258.0052795410156   
3196     99   584.1934204101562    1143.76904296875   583.8372802734375   
3197   9905  1759.2774658203125    467.811767578125  1766.5172119140625   
3198   9910   572.4219970703125   1289.455810546875   572.8764038085938   
3199   9918   580.6690063476562  1148.1163330078125   580.7485961914062   

                      v4                  v5                  v6  \
0      383.5165710449219     1